In [1]:
import collections
import numpy as np
import pygame
import random
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import keras

from scipy.misc import imresize

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.


In [6]:
class MyWrappedGame(object):
    def __init__(self):
        # run pygame in headless mode
        os.environ['SDL_VIDEODRIVER'] = 'dummy'
        
        pygame.init()
        pygame.key.set_repeat(10, 100)
        
        # set constants
        self.COLOR_WHITE = (255, 255, 255)
        self.COLOR_BLACK = (0, 0, 0)
        self.GAME_WIDTH = 400
        self.GAME_HEIGHT = 400
        self.BALL_WIDTH = 20
        self.BALL_HEIGHT = 20
        self.PADDLE_WIDTH = 50
        self.PADDLE_HEIGHT = 10
        self.GAME_FLOOR = 350
        self.GAME_CEILING = 10
        self.BALL_VELOCITY = 10
        self.PADDLE_VELOCITY = 20
        self.FONT_SIZE = 30
        self.MAX_TRIES_PER_GAME = 1
        self.CUSTOM_EVENT = pygame.USEREVENT + 1
        self.font = pygame.font.SysFont('Comic Sans MS', self.FONT_SIZE)
        
    def reset(self):
        self.frames = collections.deque(maxlen=4)
        self.game_over = False
        # initialize positions
        self.paddle_x = self.GAME_WIDTH // 2
        self.game_score = 0
        self.reward = 0
        self.ball_x = random.randint(0, self.GAME_WIDTH)
        self.ball_y = self.GAME_CEILING
        self.num_tries = 0
        
        # setup display, clock, etc
        self.screen = pygame.display.set_mode((self.GAME_WIDTH, self.GAME_HEIGHT))
        self.clock = pygame.time.Clock()
        
    def step(self, action):
        pygame.event.pump()
        
        if action == 0: # move paddle left
            self.paddle_x -= self.PADDLE_VELOCITY
            if self.paddle_x < 0:
                # bounce off the wall, go right
                self.paddle_x = self.PADDLE_VELOCITY
        elif action == 2: # move paddle right
            self.paddle_x += self.PADDLE_VELOCITY
            if self.paddle_x > self.GAME_WIDTH - self.PADDLE_WIDTH:
                # bound off the wall, go left
                self.paddle_x = self.GAME_WIDTH - self.PADDLE_WIDTH - self.PADDLE_VELOCITY
        else: # don't move paddle
            pass
        
        self.screen.fill(self.COLOR_BLACK)
        score_text = self.font.render('Score: {:d}/{:d}, Ball: {:d}'.format(self.game_score, self.MAX_TRIES_PER_GAME, self.num_tries),
                                      True, self.COLOR_WHITE)
        self.screen.blit(score_text, ((self.GAME_WIDTH - score_text.get_width()) // 2,
                                     (self.GAME_FLOOR + self.FONT_SIZE // 2)))
        
        # update ball position
        self.ball_y += self.BALL_VELOCITY
        ball = pygame.draw.rect(self.screen, self.COLOR_WHITE,
                                pygame.Rect(self.ball_x, self.ball_y, self.BALL_WIDTH, self.BALL_HEIGHT))
        # update paddle position
        paddle = pygame.draw.rect(self.screen, self.COLOR_WHITE,
                                  pygame.Rect(self.paddle_x, self.GAME_FLOOR, self.PADDLE_WIDTH, self.PADDLE_HEIGHT))
        
        # check for collision and update reward
        self.reward = 0
        if self.ball_y >= self.GAME_FLOOR - self.BALL_HEIGHT // 2:
            if ball.colliderect(paddle):
                self.reward = 1
            else:
                self.reward = -1
                
            self.game_score += self.reward
            self.ball_x = random.randint(0, self.GAME_WIDTH)
            self.ball_y = self.GAME_CEILING
            self.num_tries += 1
        
        pygame.display.flip()
        
        # save last 4 frames
        self.frames.append(pygame.surfarray.array2d(self.screen))
        
        if self.num_tries >= self.MAX_TRIES_PER_GAME:
            self.game_over = True
            
        self.clock.tick(30)
        return self.get_frames(), self.reward, self.game_over
    
    def get_frames(self):
        return np.array(list(self.frames))

In [7]:
game = MyWrappedGame()
NUM_EPOCHS = 10
for e in range(NUM_EPOCHS):
    print('Epochs: {:d}'.format(e))
    game.reset()
    input_t = game.get_frames()
    game_over = False
    while not game_over:
        action = np.random.randint(0, 3, size=1)[0]
        input_tp1, reward, game_over = game.step(action)
        print(action, reward, game_over)

Epochs: 0
1 0 False
2 0 False
1 0 False
2 0 False
1 0 False
2 0 False
1 0 False
0 0 False
1 0 False
2 0 False
0 0 False
1 0 False
2 0 False
0 0 False
0 0 False
2 0 False
2 0 False
0 0 False
1 0 False
0 0 False
1 0 False
0 0 False
1 0 False
1 0 False
0 0 False
1 0 False
0 0 False
0 0 False
1 0 False
0 0 False
1 0 False
2 0 False
1 1 True
Epochs: 1
1 0 False
0 0 False
2 0 False
2 0 False
2 0 False
2 0 False
0 0 False
1 0 False
1 0 False
2 0 False
0 0 False
1 0 False
0 0 False
1 0 False
0 0 False
0 0 False
1 0 False
1 0 False
0 0 False
1 0 False
2 0 False
2 0 False
2 0 False
2 0 False
0 0 False
1 0 False
2 0 False
2 0 False
0 0 False
2 0 False
0 0 False
0 0 False
1 -1 True
Epochs: 2
1 0 False
0 0 False
2 0 False
2 0 False
0 0 False
1 0 False
2 0 False
0 0 False
0 0 False
1 0 False
1 0 False
0 0 False
0 0 False
0 0 False
1 0 False
0 0 False
0 0 False
2 0 False
0 0 False
0 0 False
2 0 False
1 0 False
2 0 False
2 0 False
1 0 False
1 0 False
0 0 False
1 0 False
2 0 False
1 0 False
2 0 False
0

In [8]:
def preprocess_images(images):
    if images.shape[0] < 4:
        # single image
        x_t = images[0]
        x_t = imresize(x_t, (80, 80))
        x_t = x_t.astype(np.float32) / 255.
        s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)
    else:
        # 4 images
        xt_list = []
        for i in range(images.shape[0]):
            x_t = imresize(images[i], (80, 80))
            x_t = x_t.astype(np.float32) / 255.
            xt_list.append(x_t)
        s_t = np.stack((xt_list[0], xt_list[1], xt_list[2], xt_list[3]), axis=2)
    
    s_t = np.expand_dims(s_t, axis=0)
    return s_t
           
    
def get_next_batch(experience, model, num_actions, gamma, batch_size):
    batch_indices = np.random.randint(low=0, high=len(experience), size=batch_size)
    batch = [experience[i] for i in batch_indices]
    X = np.zeros((batch_size, 80, 80, 4))
    Y = np.zeros((batch_size, num_actions))
    for i in range(len(batch)):
        s_t, a_t, r_t, s_tp1, game_over = batch[i]
        X[i] = s_t
        Y[i] = model.predict(s_t)[0]
        Q_sa = np.max(model.predict(s_tp1)[0])
        if game_over:
            Y[i, a_t] = r_t
        else:
            Y[i, a_t] = r_t + gamma * Q_sa
    return X, Y

In [9]:
# create a model
model = keras.Sequential()
model.add(keras.layers.Conv2D(32, (8, 8), strides=4, kernel_initializer='normal', padding='same', input_shape=(80, 80, 4), activation='relu'))
model.add(keras.layers.Conv2D(64, (4, 4), strides=2, kernel_initializer='normal', padding='same', activation='relu'))
model.add(keras.layers.Conv2D(64, (3, 3), strides=1, kernel_initializer='normal', padding='same', activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(512, kernel_initializer='normal', activation='relu'))
model.add(keras.layers.Dense(3, kernel_initializer='normal'))

model.compile(optimizer=keras.optimizers.Adam(lr=1e-6), loss='mse')

In [10]:
# initialize parameters
DATA_DIR = 'data'
NUM_ACTIONS = 3 # number of valid actions (left, stay, right)
GAMMA = 0.99
INITIAL_EPSILON = 0.1 # starting value of epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
MEMORY_SIZE = 50000 # number of previous transitions to remember
NUM_EPOCHS_OBSERVE = 100
NUM_EPOCHS_TRAIN = 5000
BATCH_SIZE = 32
NUM_EPOCHS = NUM_EPOCHS_OBSERVE + NUM_EPOCHS_TRAIN

game = MyWrappedGame()
experience = collections.deque(maxlen=MEMORY_SIZE)
fout = open(os.path.join(DATA_DIR, 'rl-network-results.tsv'), 'w')
epsilon = INITIAL_EPSILON
num_games, num_wins = 0, 0

for e in range(NUM_EPOCHS):
    game.reset()
    loss = 0.0
    
    # get first state
    a_0 = 1 # (0 = left, 1 = stay, 2 = right)
    x_t, r_0, game_over = game.step(a_0)
    s_t = preprocess_images(x_t)
    
    while not game_over:
        s_tm1 = s_t
        # next action
        if e <= NUM_EPOCHS_OBSERVE:
            a_t = np.random.randint(low=0, high=NUM_ACTIONS, size=1)[0]
        else:
            if np.random.rand() <= epsilon:
                a_t = np.random.randint(low=0, high=NUM_ACTIONS, size=1)[0]
            else:
                q = model.predict(s_t)[0]
                a_t = np.argmax(q)
                
        # apply action, get reward
        x_t, r_t, game_over = game.step(a_t)
        s_t = preprocess_images(x_t)
        # if reward, increment num_wins
        if r_t == 1:
            num_wins += 1
        # store experience
        experience.append((s_tm1, a_t, r_t, s_t, game_over))
        
        if e > NUM_EPOCHS_OBSERVE:
            # finished observing, now start training
            # get next batch
            X, Y = get_next_batch(experience, model, NUM_ACTIONS, GAMMA, BATCH_SIZE)
            loss += model.train_on_batch(X, Y)
            
    # reduce epsilon gradually
    if epsilon > FINAL_EPSILON:
        epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / NUM_EPOCHS 
        
    print('Epoch {:04d}/{:d} | Loss {:.5f} | Win count {:d}'.format(e+1, NUM_EPOCHS, loss, num_wins))
    text = '{:04d}t{:.5f}t{:d}\n'.format(e+1, loss, num_wins)
    fout.write(text)
    if e % 100 == 0:
        model.save(os.path.join(DATA_DIR, 'rl-network.h5'), overwrite=True)
        
fout.close()
model.save(os.path.join(DATA_DIR, 'rl-network.h5'), overwrite=True)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  del sys.path[0]


Epoch 0001/5100 | Loss 0.00000 | Win count 0
Epoch 0002/5100 | Loss 0.00000 | Win count 0
Epoch 0003/5100 | Loss 0.00000 | Win count 0
Epoch 0004/5100 | Loss 0.00000 | Win count 1
Epoch 0005/5100 | Loss 0.00000 | Win count 1
Epoch 0006/5100 | Loss 0.00000 | Win count 1
Epoch 0007/5100 | Loss 0.00000 | Win count 2
Epoch 0008/5100 | Loss 0.00000 | Win count 2
Epoch 0009/5100 | Loss 0.00000 | Win count 2
Epoch 0010/5100 | Loss 0.00000 | Win count 2
Epoch 0011/5100 | Loss 0.00000 | Win count 2
Epoch 0012/5100 | Loss 0.00000 | Win count 2
Epoch 0013/5100 | Loss 0.00000 | Win count 2
Epoch 0014/5100 | Loss 0.00000 | Win count 2
Epoch 0015/5100 | Loss 0.00000 | Win count 3
Epoch 0016/5100 | Loss 0.00000 | Win count 3
Epoch 0017/5100 | Loss 0.00000 | Win count 3
Epoch 0018/5100 | Loss 0.00000 | Win count 3
Epoch 0019/5100 | Loss 0.00000 | Win count 3
Epoch 0020/5100 | Loss 0.00000 | Win count 3
Epoch 0021/5100 | Loss 0.00000 | Win count 3
Epoch 0022/5100 | Loss 0.00000 | Win count 4
Epoch 0023

Epoch 0181/5100 | Loss 0.43956 | Win count 27
Epoch 0182/5100 | Loss 0.43388 | Win count 27
Epoch 0183/5100 | Loss 0.32854 | Win count 27
Epoch 0184/5100 | Loss 0.37648 | Win count 27
Epoch 0185/5100 | Loss 0.33893 | Win count 27
Epoch 0186/5100 | Loss 0.33076 | Win count 27
Epoch 0187/5100 | Loss 0.41157 | Win count 27
Epoch 0188/5100 | Loss 0.36418 | Win count 28
Epoch 0189/5100 | Loss 0.28280 | Win count 28
Epoch 0190/5100 | Loss 0.39680 | Win count 28
Epoch 0191/5100 | Loss 0.31664 | Win count 28
Epoch 0192/5100 | Loss 0.34105 | Win count 28
Epoch 0193/5100 | Loss 0.27738 | Win count 28
Epoch 0194/5100 | Loss 0.33046 | Win count 28
Epoch 0195/5100 | Loss 0.37139 | Win count 28
Epoch 0196/5100 | Loss 0.36944 | Win count 28
Epoch 0197/5100 | Loss 0.28178 | Win count 28
Epoch 0198/5100 | Loss 0.44629 | Win count 28
Epoch 0199/5100 | Loss 0.32315 | Win count 28
Epoch 0200/5100 | Loss 0.32326 | Win count 28
Epoch 0201/5100 | Loss 0.31426 | Win count 28
Epoch 0202/5100 | Loss 0.25727 | W

Epoch 0360/5100 | Loss 0.15911 | Win count 69
Epoch 0361/5100 | Loss 0.18060 | Win count 69
Epoch 0362/5100 | Loss 0.20084 | Win count 70
Epoch 0363/5100 | Loss 0.14315 | Win count 70
Epoch 0364/5100 | Loss 0.18789 | Win count 70
Epoch 0365/5100 | Loss 0.24272 | Win count 70
Epoch 0366/5100 | Loss 0.22994 | Win count 70
Epoch 0367/5100 | Loss 0.25475 | Win count 70
Epoch 0368/5100 | Loss 0.12075 | Win count 71
Epoch 0369/5100 | Loss 0.18272 | Win count 71
Epoch 0370/5100 | Loss 0.21279 | Win count 71
Epoch 0371/5100 | Loss 0.21187 | Win count 71
Epoch 0372/5100 | Loss 0.19716 | Win count 71
Epoch 0373/5100 | Loss 0.26602 | Win count 71
Epoch 0374/5100 | Loss 0.22574 | Win count 71
Epoch 0375/5100 | Loss 0.34939 | Win count 71
Epoch 0376/5100 | Loss 0.10911 | Win count 71
Epoch 0377/5100 | Loss 0.20377 | Win count 71
Epoch 0378/5100 | Loss 0.24298 | Win count 71
Epoch 0379/5100 | Loss 0.23651 | Win count 72
Epoch 0380/5100 | Loss 0.14014 | Win count 73
Epoch 0381/5100 | Loss 0.17681 | W

Epoch 0538/5100 | Loss 0.14413 | Win count 108
Epoch 0539/5100 | Loss 0.17804 | Win count 108
Epoch 0540/5100 | Loss 0.16655 | Win count 109
Epoch 0541/5100 | Loss 0.13912 | Win count 109
Epoch 0542/5100 | Loss 0.08347 | Win count 109
Epoch 0543/5100 | Loss 0.14240 | Win count 109
Epoch 0544/5100 | Loss 0.22220 | Win count 109
Epoch 0545/5100 | Loss 0.26542 | Win count 109
Epoch 0546/5100 | Loss 0.20473 | Win count 109
Epoch 0547/5100 | Loss 0.12849 | Win count 109
Epoch 0548/5100 | Loss 0.14252 | Win count 109
Epoch 0549/5100 | Loss 0.13740 | Win count 110
Epoch 0550/5100 | Loss 0.17785 | Win count 110
Epoch 0551/5100 | Loss 0.16224 | Win count 110
Epoch 0552/5100 | Loss 0.18069 | Win count 110
Epoch 0553/5100 | Loss 0.17930 | Win count 110
Epoch 0554/5100 | Loss 0.22937 | Win count 110
Epoch 0555/5100 | Loss 0.14421 | Win count 110
Epoch 0556/5100 | Loss 0.16661 | Win count 111
Epoch 0557/5100 | Loss 0.16632 | Win count 111
Epoch 0558/5100 | Loss 0.06756 | Win count 111
Epoch 0559/51

...

Epoch 4361/5100 | Loss 0.15432 | Win count 1860
Epoch 4362/5100 | Loss 0.15013 | Win count 1861
Epoch 4363/5100 | Loss 0.15201 | Win count 1862
Epoch 4364/5100 | Loss 0.20199 | Win count 1863
Epoch 4365/5100 | Loss 0.13054 | Win count 1863
Epoch 4366/5100 | Loss 0.17892 | Win count 1864
Epoch 4367/5100 | Loss 0.12784 | Win count 1864
Epoch 4368/5100 | Loss 0.15910 | Win count 1865
Epoch 4369/5100 | Loss 0.09320 | Win count 1866
Epoch 4370/5100 | Loss 0.18730 | Win count 1866
Epoch 4371/5100 | Loss 0.19967 | Win count 1867
Epoch 4372/5100 | Loss 0.19497 | Win count 1868
Epoch 4373/5100 | Loss 0.12445 | Win count 1869
Epoch 4374/5100 | Loss 0.14793 | Win count 1870
Epoch 4375/5100 | Loss 0.15134 | Win count 1871
Epoch 4376/5100 | Loss 0.14082 | Win count 1872
Epoch 4377/5100 | Loss 0.12626 | Win count 1873
Epoch 4378/5100 | Loss 0.11682 | Win count 1874
Epoch 4379/5100 | Loss 0.12450 | Win count 1875
Epoch 4380/5100 | Loss 0.10038 | Win count 1876
Epoch 4381/5100 | Loss 0.11592 | Win cou

Epoch 4532/5100 | Loss 0.10445 | Win count 2003
Epoch 4533/5100 | Loss 0.12692 | Win count 2004
Epoch 4534/5100 | Loss 0.11425 | Win count 2005
Epoch 4535/5100 | Loss 0.12925 | Win count 2006
Epoch 4536/5100 | Loss 0.11263 | Win count 2007
Epoch 4537/5100 | Loss 0.12713 | Win count 2008
Epoch 4538/5100 | Loss 0.14466 | Win count 2008
Epoch 4539/5100 | Loss 0.19696 | Win count 2009
Epoch 4540/5100 | Loss 0.08878 | Win count 2010
Epoch 4541/5100 | Loss 0.15702 | Win count 2011
Epoch 4542/5100 | Loss 0.19268 | Win count 2012
Epoch 4543/5100 | Loss 0.11358 | Win count 2012
Epoch 4544/5100 | Loss 0.14537 | Win count 2013
Epoch 4545/5100 | Loss 0.12753 | Win count 2014
Epoch 4546/5100 | Loss 0.09633 | Win count 2015
Epoch 4547/5100 | Loss 0.13740 | Win count 2016
Epoch 4548/5100 | Loss 0.19526 | Win count 2017
Epoch 4549/5100 | Loss 0.17036 | Win count 2018
Epoch 4550/5100 | Loss 0.10584 | Win count 2019
Epoch 4551/5100 | Loss 0.14085 | Win count 2020
Epoch 4552/5100 | Loss 0.18779 | Win cou

Epoch 4703/5100 | Loss 0.11233 | Win count 2157
Epoch 4704/5100 | Loss 0.11934 | Win count 2158
Epoch 4705/5100 | Loss 0.16092 | Win count 2159
Epoch 4706/5100 | Loss 0.07602 | Win count 2160
Epoch 4707/5100 | Loss 0.09539 | Win count 2161
Epoch 4708/5100 | Loss 0.08748 | Win count 2162
Epoch 4709/5100 | Loss 0.11533 | Win count 2162
Epoch 4710/5100 | Loss 0.06515 | Win count 2162
Epoch 4711/5100 | Loss 0.09272 | Win count 2163
Epoch 4712/5100 | Loss 0.06493 | Win count 2163
Epoch 4713/5100 | Loss 0.12135 | Win count 2164
Epoch 4714/5100 | Loss 0.14231 | Win count 2165
Epoch 4715/5100 | Loss 0.09484 | Win count 2166
Epoch 4716/5100 | Loss 0.07747 | Win count 2167
Epoch 4717/5100 | Loss 0.12496 | Win count 2168
Epoch 4718/5100 | Loss 0.14854 | Win count 2168
Epoch 4719/5100 | Loss 0.10119 | Win count 2169
Epoch 4720/5100 | Loss 0.06939 | Win count 2169
Epoch 4721/5100 | Loss 0.15966 | Win count 2170
Epoch 4722/5100 | Loss 0.10493 | Win count 2171
Epoch 4723/5100 | Loss 0.16374 | Win cou

Epoch 4874/5100 | Loss 0.10416 | Win count 2311
Epoch 4875/5100 | Loss 0.08798 | Win count 2312
Epoch 4876/5100 | Loss 0.09582 | Win count 2313
Epoch 4877/5100 | Loss 0.09105 | Win count 2314
Epoch 4878/5100 | Loss 0.07854 | Win count 2315
Epoch 4879/5100 | Loss 0.07522 | Win count 2316
Epoch 4880/5100 | Loss 0.13122 | Win count 2317
Epoch 4881/5100 | Loss 0.06965 | Win count 2318
Epoch 4882/5100 | Loss 0.13476 | Win count 2319
Epoch 4883/5100 | Loss 0.09622 | Win count 2320
Epoch 4884/5100 | Loss 0.07937 | Win count 2321
Epoch 4885/5100 | Loss 0.13402 | Win count 2322
Epoch 4886/5100 | Loss 0.06368 | Win count 2323
Epoch 4887/5100 | Loss 0.14447 | Win count 2324
Epoch 4888/5100 | Loss 0.09153 | Win count 2325
Epoch 4889/5100 | Loss 0.07376 | Win count 2325
Epoch 4890/5100 | Loss 0.12244 | Win count 2326
Epoch 4891/5100 | Loss 0.09704 | Win count 2327
Epoch 4892/5100 | Loss 0.07842 | Win count 2327
Epoch 4893/5100 | Loss 0.10380 | Win count 2328
Epoch 4894/5100 | Loss 0.16061 | Win cou

Epoch 5045/5100 | Loss 0.07665 | Win count 2471
Epoch 5046/5100 | Loss 0.07317 | Win count 2472
Epoch 5047/5100 | Loss 0.09921 | Win count 2473
Epoch 5048/5100 | Loss 0.12151 | Win count 2474
Epoch 5049/5100 | Loss 0.09511 | Win count 2475
Epoch 5050/5100 | Loss 0.06702 | Win count 2475
Epoch 5051/5100 | Loss 0.09381 | Win count 2475
Epoch 5052/5100 | Loss 0.10105 | Win count 2476
Epoch 5053/5100 | Loss 0.06771 | Win count 2477
Epoch 5054/5100 | Loss 0.07997 | Win count 2478
Epoch 5055/5100 | Loss 0.10723 | Win count 2479
Epoch 5056/5100 | Loss 0.11231 | Win count 2480
Epoch 5057/5100 | Loss 0.08986 | Win count 2481
Epoch 5058/5100 | Loss 0.08551 | Win count 2482
Epoch 5059/5100 | Loss 0.07162 | Win count 2483
Epoch 5060/5100 | Loss 0.10684 | Win count 2484
Epoch 5061/5100 | Loss 0.08019 | Win count 2485
Epoch 5062/5100 | Loss 0.08710 | Win count 2486
Epoch 5063/5100 | Loss 0.09991 | Win count 2486
Epoch 5064/5100 | Loss 0.09570 | Win count 2487
Epoch 5065/5100 | Loss 0.12395 | Win cou